In [1]:
#import mariadb
import tqdm
import pickle
import pandas as pd

In [2]:
# read bitcoin conversion rates
df_btc = pd.read_csv("BTC_EUR_conversion.csv", delimiter=",")
df_btc.Date = pd.to_datetime(df_btc['Date'])
df_btc.Price = df_btc.Price.str.replace(",", "")
df_btc.Date = df_btc.Date.dt.date
df_btc.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2019-12-31,6397.4,"6,449.8","6,511.5","6,335.3",3.03K,-0.81%
1,2019-12-30,6449.8,"6,606.7","6,606.7","6,430.1",3.56K,-2.37%
2,2019-12-29,6606.7,"6,536.7","6,729.8","6,519.4",2.11K,1.07%
3,2019-12-28,6536.6,"6,485.0","6,584.8","6,473.2",1.69K,0.80%
4,2019-12-27,6485.0,"6,489.9","6,507.3","6,352.0",2.84K,-0.08%


In [7]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pymysql
df_pw = pd.read_csv("pw.csv", delimiter=",")
db_url = {
    'database': 'DarknetScrape',
    'drivername': 'mysql',
    'username': df_pw["name"][0],
    'password': df_pw["password"][0],
    'host': '127.0.0.1',
    'port': 3306,
    'query': {'charset': 'utf8'},
}

db_connection = create_engine(URL(**db_url), encoding="utf8")

c:\users\d067608\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  from ipykernel import kernelapp as app


In [9]:
query = """SELECT
	reviews.price_eur as price_eur_review,
	items.price_eur as price_eur_items,
	items.price_btc ,
	reviews.`time`,
	vendors.vendor_id ,
	reviews.item_id
FROM
	DarknetScrape.item_reviews as reviews
JOIN DarknetScrape.items as items on
	reviews.item_id = items.item_id
join DarknetScrape.vendors vendors on
	vendors.vendor_id = items.vendor_id
join DarknetScrape.market market on
	market.market_name = vendors.market
WHERE
	reviews.positive = 1
	and market.market_name = "Berlusconi Market"
"""

df = pd.read_sql(query, con=db_connection)
df.head()

,price_eur_review,price_eur_items,price_btc,time,vendor_id,item_id
0,208.82,130.0,0.015269,2019-07-07 18:40:42,b5f6b728cec773313da84887919cef33,04906d05a940
1,160.24,130.0,0.015269,2019-07-06 22:10:17,b5f6b728cec773313da84887919cef33,04906d05a940
2,125.91,130.0,0.015269,2019-06-12 17:41:47,b5f6b728cec773313da84887919cef33,04906d05a940
3,119.83,130.0,0.015269,2019-06-08 10:03:15,b5f6b728cec773313da84887919cef33,04906d05a940
4,117.43,130.0,0.015269,2019-06-06 08:36:34,b5f6b728cec773313da84887919cef33,04906d05a940


In [5]:
# convert the euro price from the reviews into bitcoin (value at the time)
df["price_btc_review"] = "0"
for i, row in df.iterrows():
    ifor_val = str(row.price_eur_review / float(df_btc[df_btc["Date"] == row.time.date()]["Price"].iloc[0]))
    df.at[i,'price_btc_review'] = ifor_val
df.head()

,price_eur_review,price_eur_items,price_btc,time,vendor_id,item_id,price_btc_review
0,208.82,130.0,0.015269,2019-07-07 18:40:42,b5f6b728cec773313da84887919cef33,04906d05a940,0.020427688214118014
1,160.24,130.0,0.015269,2019-07-06 22:10:17,b5f6b728cec773313da84887919cef33,04906d05a940,0.01599552796023079
2,125.91,130.0,0.015269,2019-06-12 17:41:47,b5f6b728cec773313da84887919cef33,04906d05a940,0.017387041537781703
3,119.83,130.0,0.015269,2019-06-08 10:03:15,b5f6b728cec773313da84887919cef33,04906d05a940,0.01708123672544296
4,117.43,130.0,0.015269,2019-06-06 08:36:34,b5f6b728cec773313da84887919cef33,04906d05a940,0.01698118664410799


In [6]:
df.to_csv("reviews_price_btc.csv")